<a href="https://colab.research.google.com/github/EdithOroche/IA/blob/main/Laboratorio01_unidad_II_PI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolución 2D (imágenes RGB de 3 canales)



## 1. Convulsión 2D: notación y dimensiones

Usaremos la notación estándar de *deep learning* (PyTorch):

- Entrada: un *batch* de imágenes con forma **`(N, C_in, H_in, W_in)`**.
  - `N`: tamaño de lote.
  - `C_in`: canales de entrada. Para imágenes RGB, `C_in = 3`.
  - `H_in`, `W_in`: alto y ancho de la imagen.
- Filtros (pesos): **`(C_out, C_in, K_h, K_w)`**.
  - `C_out`: número de mapas de salida (canales de salida).
  - `K_h`, `K_w`: alto y ancho del *kernel* (filtro).
- Hiperparámetros:
  - *Padding*: `P_h`, `P_w` (número de ceros que se “agregan” en los bordes alto y ancho).
  - *Stride*: `S_h`, `S_w` (desplazamiento del filtro vertical y horizontal).
  - *(Opcional)* **Dilatación**: `D_h`, `D_w` (aumenta el “salto” entre posiciones del *kernel*). Aquí la consideraremos **1** para centrarnos en lo pedido.

> En PyTorch, la **convolución 2D** estándar mezcla los `C_in` canales de entrada con cada filtro (que también tiene `C_in` canales) y produce `C_out` mapas. El tamaño espacial de cada mapa está dado por las fórmulas de abajo.

---

## 2. Fórmula del tamaño de salida (sin dilatación; `D_h = D_w = 1`)

Para una **convolución 2D** con *padding* `(P_h, P_w)`, *stride* `(S_h, S_w)` y *kernel* `(K_h, K_w)`, el tamaño espacial de la salida es:

$$
H_{\text{out}} = \left\lfloor \frac{H_{\text{in}} + 2P_h - K_h}{S_h} \right\rfloor + 1,\qquad
W_{\text{out}} = \left\lfloor \frac{W_{\text{in}} + 2P_w - K_w}{S_w} \right\rfloor + 1.
$$

Y el **número de canales de salida** es `C_out`. Por lo tanto, la forma final es:

$$
(N,\; C_{\text{out}},\; H_{\text{out}},\; W_{\text{out}}).
$$

**Condiciones de validez** (para que \(H_{\text{out}}, W_{\text{out}}\) sean positivos):
$$
H_{\text{in}} + 2P_h - K_h \ge 0,\qquad W_{\text{in}} + 2P_w - K_w \ge 0.
$$

Si la división no es exacta, PyTorch usa el **piso** (*floor*).

---

## 3. ¿Qué hace *Padding* y *Stride*?

- **Padding** “amplía” virtualmente la imagen con ceros en los bordes para permitir más posiciones del *kernel*. A mayor `P_h`/`P_w`, mayor $H_{\text{out}}, W_{\text{out}}$ (hasta cierto punto).
- **Stride** controla cuánto se “salta” al mover el *kernel*. A mayor `S_h`/`S_w`, **menor** $H_{\text{out}}, W_{\text{out}}$.

Casos comunes:
- **`padding='valid'`** (o `P_h=P_w=0`): no hay ceros añadidos. $\Rightarrow$ salida “más pequeña”.
- **`padding='same'`** (PyTorch lo soporta desde v2.0 en adelante para `Conv2d`): el *framework* elige `P_h,P_w` para que, con `S=1`, se cumpla $H_{\text{out}}=H_{\text{in}}$ y $W_{\text{out}}=W_{\text{in}}$ (o lo más cercano posible con *floor*).

---

## 4. Convolución con **3 canales de entrada**

Para una imagen RGB $C_{\text{in}}=3$:
- Cada filtro tiene forma $(C_{\text{in}}=3,\; K_h,\; K_w)$.
- El resultado agrega las contribuciones de los 3 canales (con sus pesos respectivos) y produce un mapa. Con `C_out` filtros distintos, se obtienen `C_out` mapas.

**Importante:** El tamaño espacial $(H_{\text{out}}, W_{\text{out}})$ **no depende** de `C_in`, sino de $(H_{\text{in}}, W_{\text{in}})$, $(K_h, K_w)$, $(P_h, P_w)$ y $(S_h, S_w)$. `C_in` y `C_out` solo afectan la **profundidad** (canales) y el número de filtros/aprendizaje de parámetros.

---

## 5. Extensión con **dilatación** (referencia)

Si se usara dilatación \((D_h, D_w)\), la fórmula general es:

$$
H_{\text{out}} = \left\lfloor \frac{H_{\text{in}} + 2P_h - D_h\,(K_h-1) - 1}{S_h} \right\rfloor + 1,\qquad
W_{\text{out}} = \left\lfloor \frac{W_{\text{in}} + 2P_w - D_w\,(K_w-1) - 1}{S_w} \right\rfloor + 1.
$$

En este cuaderno asumiremos $D_h=D_w=1$ (sin dilatación) para centrarnos en lo solicitado.



## 6. Experimentos en PyTorch: verificación de tamaños

A continuación, verificaremos la teoría creando capas `nn.Conv2d` y comparando la forma calculada con la reportada por PyTorch. **Si en tu entorno no está instalado PyTorch**, estos bloques no se ejecutarán; en ese caso, puedes subir el mismo notebook a **Google Colab** o a un entorno con PyTorch preinstalado.


In [ ]:
# === Setup: importaciones y utilidades ===
try:
    import torch
    import torch.nn as nn
    TORCH_OK = True
except Exception as e:
    TORCH_OK = False
    print("PyTorch no está disponible en este entorno. Puedes ejecutar este notebook en Google Colab o un entorno con PyTorch.\nDetalle:", e)

import math
import itertools
import pandas as pd

def conv2d_out_hw(H_in, W_in, K_h, K_w, P_h, P_w, S_h, S_w):
    """Calcula (H_out, W_out) para conv2d sin dilatación (D=1)."""
    H_out = math.floor((H_in + 2*P_h - K_h)/S_h) + 1
    W_out = math.floor((W_in + 2*P_w - K_w)/S_w) + 1
    return H_out, W_out



### 6.1. Caso simple con RGB (3 canales)

Probemos con una imagen `H_in=128, W_in=96`, `C_in=3`, un *kernel* `K=3×3`, *padding* `P=1`, *stride* `S=1`. Por teoría, con `K=3` y `P=1`, `S=1` se cumple \(H_{out}=H_{in}\) y \(W_{out}=W_{in}\).


In [ ]:
# Parámetros
N, C_in, H_in, W_in = 4, 3, 128, 96
K_h, K_w = 3, 3
P_h, P_w = 1, 1
S_h, S_w = 1, 1
C_out = 16

# Cálculo teórico
H_out_th, W_out_th = conv2d_out_hw(H_in, W_in, K_h, K_w, P_h, P_w, S_h, S_w)
print("Teoría -> H_out, W_out:", (H_out_th, W_out_th))

if TORCH_OK:
    # Tensor de entrada
    x = torch.randn(N, C_in, H_in, W_in)
    # Capa conv2d
    conv = nn.Conv2d(in_channels=C_in, out_channels=C_out,
                     kernel_size=(K_h, K_w), stride=(S_h, S_w), padding=(P_h, P_w), bias=False)
    y = conv(x)
    print("PyTorch -> y.shape:", tuple(y.shape))
else:
    print("PyTorch no disponible: omitiendo ejecución de Conv2d.")



### 6.2. Rejilla de pruebas (múltiples combinaciones)

Generaremos una tabla comparando la **forma teórica** con la **medida por PyTorch** para distintas combinaciones de `K`, `P` y `S` (manteniendo `C_in=3`), y distintos tamaños de entrada.


In [ ]:
rows = []
Hs = [32, 33]
Ws = [32, 35]
Ks = [(1,1), (3,3), (5,5)]
Ps = [(0,0), (1,1), (2,2)]
Ss = [(1,1), (2,2), (3,3)]
C_in = 3
C_out = 8
N = 2

for H_in, W_in in itertools.product(Hs, Ws):
    for (K_h, K_w) in Ks:
        for (P_h, P_w) in Ps:
            for (S_h, S_w) in Ss:
                H_out_th, W_out_th = conv2d_out_hw(H_in, W_in, K_h, K_w, P_h, P_w, S_h, S_w)
                pyt_shape = None
                ok = None
                err = None
                if TORCH_OK and H_out_th > 0 and W_out_th > 0:
                    try:
                        x = torch.randn(N, C_in, H_in, W_in)
                        conv = nn.Conv2d(C_in, C_out, (K_h, K_w), (S_h, S_w), (P_h, P_w), bias=False)
                        y = conv(x)
                        pyt_shape = tuple(y.shape)
                        ok = (pyt_shape == (N, C_out, H_out_th, W_out_th))
                    except Exception as e:
                        err = str(e)[:120]
                rows.append({
                    "H_in": H_in, "W_in": W_in,
                    "K": f"{K_h}x{K_w}", "P": f"{P_h},{P_w}", "S": f"{S_h},{S_w}",
                    "H_out_th": H_out_th, "W_out_th": W_out_th,
                    "PyTorch_shape": pyt_shape, "Match?": ok, "Err": err
                })

df = pd.DataFrame(rows)
from caas_jupyter_tools import display_dataframe_to_user
display_dataframe_to_user("Comparación teoría vs PyTorch (rejilla)", df)
df.head(10)  # para mostrar algo en la salida textual también



### 6.3. `padding='same'` y `padding='valid'`

- `padding='valid'` equivale a `P=0`. La fórmula se reduce a: $\;H_{\text{out}} = \lfloor (H_{\text{in}} - K_h)/S_h \rfloor + 1$.
- `padding='same'` intenta mantener $H_{\text{out}}\approx H_{\text{in}}$ y $W_{\text{out}}\approx W_{\text{in}}$ cuando `S=1`. Para `S>1`, se cumple la aproximación
$$
H_{\text{out}} \approx \left\lceil \frac{H_{\text{in}}}{S_h} \right\rceil,\quad
W_{\text{out}} \approx \left\lceil \frac{W_{\text{in}}}{S_w} \right\rceil,
$$
según cómo PyTorch reparte el *padding*.


In [ ]:
tests = [
    dict(H_in=31, W_in=35, K=(3,3), S=(1,1), padding='valid'),
    dict(H_in=31, W_in=35, K=(3,3), S=(1,1), padding='same'),
    dict(H_in=31, W_in=35, K=(5,5), S=(2,2), padding='same'),
]

def out_hw_valid(H_in, W_in, K_h, K_w, S_h, S_w):
    H_out = math.floor((H_in - K_h)/S_h) + 1
    W_out = math.floor((W_in - K_w)/S_w) + 1
    return H_out, W_out

for t in tests:
    H_in, W_in = t['H_in'], t['W_in']
    K_h, K_w = t['K']
    S_h, S_w = t['S']
    padding = t['padding']
    print("\nCaso:", t)
    if padding == 'valid':
        print("Teoría (valid) ->", out_hw_valid(H_in, W_in, K_h, K_w, S_h, S_w))
    if TORCH_OK:
        x = torch.randn(1, 3, H_in, W_in)
        conv = nn.Conv2d(3, 4, (K_h, K_w), (S_h, S_w), padding=padding, bias=False)
        y = conv(x)
        print("PyTorch -> y.shape:", tuple(y.shape))
    else:
        print("PyTorch no disponible.")



### 6.4. Nota sobre mezcla de canales (`C_in=3`) y número de filtros (`C_out`)

Cada filtro de `Conv2d` tiene dimensión $(C_{\text{in}}, K_h, K_w)$. En una imagen RGB:
- Se multiplican y suman los 3 mapas de entrada con sus pesos por posición del *kernel* → 1 mapa de salida por filtro.
- Usar `C_out` filtros produce `C_out` mapas, por lo que la salida tiene forma $(N, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})$.

Esto **no** cambia las fórmulas de $H_{\text{out}}, W_{\text{out}}$, pero sí el *número* de canales resultante.



### 6.5. Prueba aleatoria de consistencia

Probamos `M` configuraciones aleatorias y verificamos que la fórmula coincida con PyTorch cuando la capa es válida (sin dimensiones negativas).


In [ ]:
import random
M = 20
ok_count = 0
checked = 0

if TORCH_OK:
    for _ in range(M):
        H_in = random.randint(16, 64)
        W_in = random.randint(16, 64)
        C_in = 3
        C_out = random.choice([4, 8, 16])
        K_h = random.choice([1,3,5])
        K_w = random.choice([1,3,5])
        P_h = random.randint(0, 3)
        P_w = random.randint(0, 3)
        S_h = random.choice([1,2,3])
        S_w = random.choice([1,2,3])
        H_out_th, W_out_th = conv2d_out_hw(H_in, W_in, K_h, K_w, P_h, P_w, S_h, S_w)
        if H_out_th <= 0 or W_out_th <= 0:
            continue  # configuración inválida
        x = torch.randn(2, C_in, H_in, W_in)
        conv = nn.Conv2d(C_in, C_out, (K_h, K_w), (S_h, S_w), (P_h, P_w), bias=False)
        y = conv(x)
        ok = tuple(y.shape) == (2, C_out, H_out_th, W_out_th)
        ok_count += int(ok)
        checked += 1

    print(f"Probadas {checked} configuraciones válidas; coincidencias teoría==PyTorch: {ok_count}/{checked}")
else:
    print("PyTorch no disponible; omitiendo prueba aleatoria.")



## 7. Conclusiones

- Para **convolución 2D** sin dilatación, las dimensiones espaciales de salida vienen dadas por:

$$
H_{out} = \left\lfloor \frac{H_{in} + 2P_h - K_h}{S_h} \right\rfloor + 1, \quad
W_{out} = \left\lfloor \frac{W_{in} + 2P_w - K_w}{S_w} \right\rfloor + 1.
$$

- El número de canales de salida es `C_out`, determinado por la cantidad de filtros.
- `padding='valid'` ($P=0$) reduce el tamaño; `padding='same'` conserva tamaño para `S=1` y lo aproxima a `ceil(H_in/S)` y `ceil(W_in/S)` para `S>1`.
- Las pruebas con PyTorch muestran consistencia entre la **fórmula teórica** y la **implementación práctica**.
